Библиотеки

In [ ]:
import os
import typing as tp
from itertools import product
import warnings

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

import cv2
import tifffile as tiff

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Пути и прочие константы

In [ ]:
IMG_PATH = '/home/jupyter-igor_busov/Seed/Hyperspectral/data/session_000/'
PROJECT_PATH = '/home/jupyter-igor_busov/Seed'
CALIBR_WHITE_PATH = '/home/jupyter-igor_busov/Seed/Hyperspectral/data/calibr/session_000_024_snapshot_cube.tiff'
CALIBR_BLACK_CAP_PATH = '/home/jupyter-igor_busov/Seed/Hyperspectral/data/calibr/session_000_025_snapshot_cube.tiff'
CALIBR_BLACK_BACKGROUND_PATH = '/home/jupyter-igor_busov/Seed/Hyperspectral/data/calibr/session_000_026_snapshot_cube.tiff'
PURPLE_IMG = '/home/jupyter-igor_busov/Seed/Hyperspectral/data/session_000/session_000_046_snapshot_cube.tiff'

$\textbf{Код}$

класс, описывающий гиперспектральное изображение с основными используемыми в этой работе методами 

In [ ]:
class Hyper_Img:
    """
    Hyperspectral image with basic methods
    """
    
    def __init__(self, path: str, threshold_value: float = 25, 
                 savgol_par: tp.Tuple[int] = (9, 3)) -> None:
        self.savgol_par = savgol_par
        self._threshold_value = threshold_value
        self.path = path
        self.img = self._get_tiff()
        self.widht = self.img.shape[0]
        self.height = self.img.shape[1]
        self.mutation = self._get_mutation()
        self.pixels =  self._get_pixels()
        self.medians = self._get_medians()
    
    @staticmethod
    def wave_len(x: int, step: int = 4, begin_wave_len: int = 450) -> int:
        return int((x - begin_wave_len) // step)
    
    def _get_tiff(self) -> None:
        img = tiff.imread(self.path)
        bl_img = tiff.imread(CALIBR_BLACK_BACKGROUND_PATH)
        new_img = np.where(bl_img > img, 0, img - bl_img)
        wh_img = tiff.imread(CALIBR_WHITE_PATH)
        return new_img /(wh_img - bl_img)
    
    def _get_mutation(self) -> str:
        if not set(IMG_PATH.split('/')).issubset(set(self.path.split('.')[0].split('/'))):
            raise NameError('Error in path')
        
        number = int(self.path.split('/')[-1].split('.')[0].split('_')[2])
        
        if number < 25 and number > 0:
            return '-4'  
        elif number > 24 and number < 27:
            return '-1' 
        elif number > 26 and number < 41:
            return 'wt' 
        elif number > 40 and number < 45:
            return '-1'
        elif number == 45:
            return 'light'
        elif number == 46:
            return 'purple'
        elif number == 47:
            return 'blue'
        else:
            return 'control'
        
    def _get_pixels(self) -> tp.List[tp.Tuple[int, int]]:
        return [(x, y) for x, y in product(range(self.widht), range(self.height)) 
                if self.threshold_bgr[x,y] != 0]
    
    def _get_medians(self) -> np.array:
        medians: tp.List[float] = list()
        for i in range(self.img.shape[2]):
            medians.append(np.median(np.array([self.img[p[0]][p[1]][i] for p in self.pixels])))
        return savgol_filter(np.array(medians), *self.savgol_par)
    
    def is_purple(self) -> bool:
        return self.path == PURPLE_IMG
 
    @property
    def bgr(self) -> np.array:
        
        #To accurately display colors, you need to choose constants
        
        im_r = self.img[:,:,Hyper_Img.wave_len(630)]
        im_g = self.img[:,:,Hyper_Img.wave_len(510)]
        im_b = self.img[:,:,Hyper_Img.wave_len(450)]
    
        im_r = (im_r / im_r.max())*255
        im_g = (im_g / im_g.max())*255
        im_b = (im_b / im_b.max())*255
    
        im_r = np.clip(im_r,0,255).astype(np.uint8)
        im_g = np.clip(im_g,0,255).astype(np.uint8)
        im_b = np.clip(im_b,0,255).astype(np.uint8)
    
        im_bgr = np.zeros((self.widht, self.height, 3), dtype = np.uint8)
        im_bgr[:,:,0] = im_b
        im_bgr[:,:,1] = im_g
        im_bgr[:,:,2] = im_r
    
        return im_bgr
    
    @property
    def threshold_bgr(self) -> np.array:
        im_black = cv2.cvtColor(self.bgr, cv2.COLOR_BGR2GRAY)
        _, im_thr = cv2.threshold(im_black, self._threshold_value, 255, cv2.THRESH_BINARY)
        return im_thr
    
    def __repr__(self) -> str:
        fig, axes = plt.subplots(1, 2)
        
        axes[0].imshow(self.bgr, cmap = 'gray')
        axes[0].set_title('rgb visualization')
        
        axes[1].imshow(self.threshold_bgr, cmap = 'gray')
        axes[1].set_title('segmentation')
        
        return f'mutation: {self.mutation}'
     

Примеры изображений (визуализация не точно передает цвета)

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_046_snapshot_cube.tiff')

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_045_snapshot_cube.tiff')

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_047_cube.tiff')

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_017_snapshot_cube.tiff')

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_025_snapshot_cube.tiff')

In [ ]:
Hyper_Img(IMG_PATH + 'session_000_027_snapshot_cube.tiff')

Функция, возвращающая необходимые имена в директории

In [ ]:
def all_tiff_cube_img(path: str) -> tp.List[str]:
    img_names: tp.List[str] = list()
    for dirname, _, filenames in os.walk(path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'tiff':
                continue
            if name.split('.')[0].split('_')[-1] != 'cube':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names 

считываем все необходимые изображения 

In [ ]:
#hyper_imgs - list with all hyperspectral images
hyper_imgs: tp.List[Hyper_Img] = [Hyper_Img(name) for name in all_tiff_cube_img(IMG_PATH)
                                  if Hyper_Img(name).mutation != 'control']    

$\textbf{Графики медиан для каждого канала}$

In [ ]:
def get_all_medians(hyper_imgs: tp.List[Hyper_Img], with_purple: bool = True) -> pd.DataFrame:
    """
    create DataFrame for graphics
    """
    
    x_axis: tp.List[int] = list(np.arange(0,138)*4 + 450)
    points: tp.List[tp.Tuple[float, float, int, str]] = list()
    if not with_purple:
        pur_img = Hyper_Img(PURPLE_IMG)
    
    for sample_number, sample in enumerate(hyper_imgs):
        
        if sample.path == PURPLE_IMG and not with_purple:
            continue
        
        if sample.mutation == 'control':
            continue
        
        if with_purple:
            point = zip(x_axis, sample.medians)
        else:
            point = zip(x_axis, sample.medians - pur_img.medians)
        
        for p in point:
            points.append([p[0], p[1], sample_number, sample.mutation])
        

    return pd.DataFrame(points, columns = ['Wavelength', 'Median', 'Sample', 'Mutation'])      

In [ ]:
df_all_med = get_all_medians(hyper_imgs)
df_all_med.sample(7)

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(data=df_all_med, x='Wavelength', y='Median', hue='Mutation')

$\textbf{Графики медиан для каждого канала с вычетем вектора медиан фиолетовых зерен}$

In [ ]:
df_med_without_pur = get_all_medians(hyper_imgs, with_purple = False)
df_med_without_pur.sample(7)

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(data=df_med_without_pur, x='Wavelength', y='Median', hue='Mutation')

$\textbf{PCA}$

In [ ]:
#Пайплайн
pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=5))])

In [ ]:
def get_med_df(hyper_imgs: tp.List[Hyper_Img]) -> pd.DataFrame:
    """
    create DataFrame for PCA
    """
        
    return pd.DataFrame([list(sample.medians) + [sample.mutation] for sample in hyper_imgs 
                         if sample.mutation != 'control'], 
                        columns = list(np.arange(0,138)*4 + 450) + ['Mutation'])    

In [ ]:
df = get_med_df(hyper_imgs)
df.sample(7)

In [ ]:
X = df.drop(['Mutation'], axis = 1)
X.head()

In [ ]:
y = df[['Mutation']]
y.head()

обучение и процент дисперсии для каждой компаненты

In [ ]:
X = pipe.fit_transform(X)
pipe['pca'].explained_variance_ratio_

визуализация

In [ ]:
warnings.simplefilter('ignore')
new_arr = np.array(tuple(zip(X[:,:2], y['Mutation'])))
lst_of_value = [(new_arr[i][0][0], new_arr[i][0][1], new_arr[i][1]) for i, _ in enumerate(new_arr)]
pd.DataFrame(lst_of_value, columns = ['1', '2', 'Mutation']).sample(7)

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(data=pd.DataFrame(lst_of_value, 
                                  columns=['1', '2', 'Mutation']), x='1', y='2', hue='Mutation')

In [ ]:
new_arr = np.array(tuple(zip(X[:,:2], y['Mutation'])))
lst_of_value = [(new_arr[i][0][0], new_arr[i][0][1], new_arr[i][1]) for i, _ in enumerate(new_arr) 
                if new_arr[i][1] != 'purple']
pd.DataFrame(lst_of_value, columns = ['1', '2', 'Mutation']).sample(7)

визуализация без фиолетовых зерен

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(data=pd.DataFrame(lst_of_value, 
                                  columns=['1', '2', 'Mutation']), x='1', y='2', hue='Mutation')